# Sum Frame Sudoku puzzle

Fill in numbers from 1 to 9 so that each row, column and 3x3 block contains each number exactly once.
Numbers in the outside frame equal the sum of the first three numbers in the corresponding row or column
in the given direction.

Find more examples [here](http://frame-sudoku.blogspot.com)

Imports

In [83]:
from ortools.constraint_solver import pywrapcp
from itertools import product

Pretty-print board

In [84]:
def pretty_print(board):
    for i in range(len(board)):
        for j in range(len(board)):
            print("[{}] ".format(board[i][j].Value()), end='')
        print("\n")
    print("\n\n")

Sudoku puzzle from slides

In [85]:
top = [15, 18, 12, 11, 21, 13, 15, 17, 13]
right = [22, 8, 15, 22, 12, 11, 15, 13, 17]
bottom = [15, 9, 21, 10, 16, 19, 13, 15, 17]
left = [8, 15, 22, 11, 13, 21, 18, 19, 8]

And another example (commented out)

In [86]:
# top = [21, 12, 12, 13, 14, 18, 10, 19, 16]
# right = [20, 15, 10, 22, 8, 15, 17, 15, 13]
# bottom = [17, 9, 19, 18, 13, 14, 23, 15, 7]
# left = [12, 12, 21, 14, 14, 17, 14, 9, 22]

Create constraint solver

In [87]:
solver = pywrapcp.Solver("Sum Frame Sudoku")

In [88]:
# Type your model here ...

Configure solver

In [89]:
cell_size = 3
board_size = cell_size * cell_size
board_indices = list(product(range(board_size), repeat=2))  # tuples (i, j) for all board indices
cell_indices = list(product(range(cell_size), repeat=2))  # tuples (i, j) for all cell indices
board = [[solver.IntVar(1, board_size) for _j in range(board_size)] for _i in range(board_size)]

Constraints

In [90]:
for i in range(board_size):
    solver.Add(solver.AllDifferent([board[i][j] for j in range(board_size)]))  # Rows
    solver.Add(solver.AllDifferent([board[j][i] for j in range(board_size)]))  # Columns

Each 3x3 sub-matrix contains only different values

In [91]:
for i, j in cell_indices:
    solver.Add(solver.AllDifferent(
        [board[i * cell_size + di][j * cell_size + dj] for di in range(cell_size) for dj in range(cell_size)]))

Preconditions

In [92]:
# Top summe
for i in range(board_size):
    solver.Add(solver.Sum([board[j][i] for j in range(cell_size)]) == top[i])
# left summe
for i in range(board_size):
    solver.Add(solver.Sum([board[i][j] for j in range(cell_size)]) == left[i])
# bottom summe
for i in range(board_size):
    solver.Add(solver.Sum([board[(board_size - 1) -j][i] for j in range(cell_size)]) == bottom[i])

# right summe
for i in range(board_size):
    solver.Add(solver.Sum([board[i][(board_size -1) - j] for j in range(cell_size)]) == right[i])

In [93]:
# Replace this by a list of decision variables


db = solver.Phase([board[i][j] for i, j in board_indices], solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

Start solver

In [94]:
solver.NewSearch(db)
while solver.NextSolution():
    pretty_print(board)

[1] [4] [3] [2] [8] [5] [9] [6] [7] 

[8] [5] [2] [6] [9] [7] [4] [3] [1] 

[6] [9] [7] [3] [4] [1] [2] [8] [5] 

[2] [3] [6] [7] [1] [4] [8] [5] [9] 

[4] [8] [1] [9] [5] [6] [3] [7] [2] 

[9] [7] [5] [8] [2] [3] [6] [1] [4] 

[7] [2] [9] [1] [3] [8] [5] [4] [6] 

[5] [6] [8] [4] [7] [2] [1] [9] [3] 

[3] [1] [4] [5] [6] [9] [7] [2] [8] 






Cleanup

In [95]:
solver.EndSearch()

Print solver information

In [96]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))

Solutions: 1
Runtime:   10967ms
Failures:  91
Branches:  182 
